# Financial ML System - SVM Training

This notebook trains the SVM classifier for market regime detection.

## 1. Setup

In [ ]:
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('/content/financial-ml-system')
sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.constants import DATA_DIR, MODELS_DIR, REGIME_NAMES
from src.utils.config_loader import config
from src.utils.helpers import time_series_split

print("Setup complete")

## 2. Load Features

In [ ]:
ticker = config.get('data.default_ticker', 'SPY')
features_file = DATA_DIR / 'processed' / f"{ticker}_features.csv"

if not features_file.exists():
    raise FileNotFoundError(f"Run 02_feature_engineering.ipynb first")

data = pd.read_csv(features_file, index_col=0, parse_dates=True)
print(f"Loaded {len(data)} rows with {len(data.columns)} columns")
data.head()

## 3. Prepare Data for SVM

In [ ]:
# Select features for SVM
feature_cols = [
    'SMA_Ratio_5', 'SMA_Ratio_20', 'SMA_Ratio_50',
    'RSI', 'MACD', 'MACD_Hist',
    'BB_Width', 'BB_Position',
    'ATR_Pct', 'Volatility',
    'Volume_Ratio', 'ROC'
]

# Check if all features exist
missing = [col for col in feature_cols if col not in data.columns]
if missing:
    print(f"Warning: Missing features: {missing}")
    feature_cols = [col for col in feature_cols if col in data.columns]

X = data[feature_cols].values
y = data['Regime'].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nClass distribution:")
for regime, name in REGIME_NAMES.items():
    count = (y == regime).sum()
    print(f"{name}: {count} ({count/len(y)*100:.1f}%)")

## 4. Train-Test Split

In [ ]:
# Time series split
train_size = config.get('data.train_test_split', 0.8)
split_idx = int(len(X) * train_size)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining date range: {data.index[0]} to {data.index[split_idx-1]}")
print(f"Test date range: {data.index[split_idx]} to {data.index[-1]}")

## 5. Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled")
print(f"Training set mean: {X_train_scaled.mean():.4f}")
print(f"Training set std: {X_train_scaled.std():.4f}")

## 6. Hyperparameter Tuning

In [ ]:
# Parameter grid
param_grid = {
    'C': config.get('models.svm.C_options', [0.1, 1, 10, 100]),
    'kernel': config.get('models.svm.kernel_options', ['rbf', 'poly', 'linear']),
    'gamma': ['scale', 'auto']
}

print("Parameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=config.get('models.svm.cv_folds', 5))

# Grid search
print("\nStarting grid search...")
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=tscv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("\nBest parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nBest cross-validation score: {grid_search.best_score_:.4f}")

## 7. Train Final Model

In [ ]:
# Get best model
best_svm = grid_search.best_estimator_

# Training accuracy
train_pred = best_svm.predict(X_train_scaled)
train_accuracy = accuracy_score(y_train, train_pred)

# Test accuracy
test_pred = best_svm.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

## 8. Evaluation

In [ ]:
# Classification report
print("Classification Report (Test Set):")
print("=" * 60)
print(classification_report(y_test, test_pred, target_names=list(REGIME_NAMES.values())))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, test_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=list(REGIME_NAMES.values()),
            yticklabels=list(REGIME_NAMES.values()))
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'svm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Prediction confidence
test_proba = best_svm.decision_function(X_test_scaled)
test_confidence = np.max(test_proba, axis=1) if len(test_proba.shape) > 1 else np.abs(test_proba)

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(test_confidence, bins=50, edgecolor='black', alpha=0.7)
ax.set_title('SVM Prediction Confidence Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Confidence Score')
ax.set_ylabel('Frequency')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'svm_confidence.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Mean confidence: {test_confidence.mean():.4f}")
print(f"Std confidence: {test_confidence.std():.4f}")

## 9. Feature Importance (Linear Kernel Only)

In [ ]:
if grid_search.best_params_['kernel'] == 'linear':
    # Get feature importance from linear kernel
    importance = np.abs(best_svm.coef_).mean(axis=0)
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': importance
    }).sort_values('Importance', ascending=False)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(feature_importance['Feature'], feature_importance['Importance'])
    ax.set_title('Feature Importance (Linear SVM)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Importance')
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'results' / 'svm_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Top 5 features:")
    print(feature_importance.head())
else:
    print(f"Feature importance not available for {grid_search.best_params_['kernel']} kernel")

## 10. Save Model

In [ ]:
# Save SVM model
model_dir = MODELS_DIR / 'svm'
model_dir.mkdir(parents=True, exist_ok=True)

svm_path = model_dir / 'svm_classifier.pkl'
scaler_path = model_dir / 'scaler.pkl'

joblib.dump(best_svm, svm_path)
joblib.dump(scaler, scaler_path)

# Save feature names
feature_info = {
    'feature_names': feature_cols,
    'train_accuracy': float(train_accuracy),
    'test_accuracy': float(test_accuracy),
    'best_params': grid_search.best_params_
}

import json
with open(model_dir / 'model_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print(f"Model saved to: {svm_path}")
print(f"Scaler saved to: {scaler_path}")
print(f"Model info saved to: {model_dir / 'model_info.json'}")

## 11. Create SVM Module

In [ ]:
# Save SVM module
svm_module = '''"""SVM classifier module."""

import numpy as np
import pandas as pd
import joblib
from pathlib import Path
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

class SVMRegimeClassifier:
    """SVM-based market regime classifier."""
    
    def __init__(self, model_path: Path = None):
        self.model = None
        self.scaler = None
        if model_path:
            self.load(model_path)
    
    def train(self, X, y, param_grid=None):
        """Train SVM classifier."""
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)
        
        if param_grid:
            tscv = TimeSeriesSplit(n_splits=5)
            grid_search = GridSearchCV(SVC(random_state=42), param_grid, 
                                      cv=tscv, n_jobs=-1)
            grid_search.fit(X_scaled, y)
            self.model = grid_search.best_estimator_
        else:
            self.model = SVC(random_state=42)
            self.model.fit(X_scaled, y)
    
    def predict(self, X):
        """Predict regime."""
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)
    
    def predict_proba(self, X):
        """Predict regime probabilities."""
        X_scaled = self.scaler.transform(X)
        return self.model.decision_function(X_scaled)
    
    def save(self, path: Path):
        """Save model."""
        joblib.dump(self.model, path / 'svm_classifier.pkl')
        joblib.dump(self.scaler, path / 'scaler.pkl')
    
    def load(self, path: Path):
        """Load model."""
        self.model = joblib.load(path / 'svm_classifier.pkl')
        self.scaler = joblib.load(path / 'scaler.pkl')
'''

with open(PROJECT_ROOT / 'src' / 'models' / 'svm_classifier.py', 'w') as f:
    f.write(svm_module)

print("Created: src/models/svm_classifier.py")

## SVM Training Complete

Next: Open `04_nn_training.ipynb`